In [1]:
! pip install opencv-python

In [2]:
import cv2
cv2.__version__

'4.9.0'

In [3]:
# 학습용 데이터 불러오기
from tensorflow.keras.datasets import mnist
(train_X, train_y), (test_X, test_y) = mnist.load_data()
train_X = train_X.reshape(-1, 28, 28, 1) / 255.0
test_X = test_X.reshape(-1, 28, 28, 1) / 255.0
# 인공신경망 모델 만들고 학습하기
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(hidden_layer_sizes=(512,256,128), max_iter=20, verbose=True)
model.fit(train_X.reshape(-1, 784), train_y)
# 모델 저장하기

Iteration 1, loss = 0.25970685
Iteration 2, loss = 0.09157059
Iteration 3, loss = 0.05744923
Iteration 4, loss = 0.04101410
Iteration 5, loss = 0.02988181
Iteration 6, loss = 0.02243895
Iteration 7, loss = 0.02153478
Iteration 8, loss = 0.01952730
Iteration 9, loss = 0.01631602
Iteration 10, loss = 0.01302968


c:\Users\hangil\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPClassifier(hidden_layer_sizes=(512, 256, 128), max_iter=20, verbose=True)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Dropout, Flatten

In [7]:
model = Sequential()
model.add(Input(shape=(28,28,1)))

model.add(Conv2D(32, 3, activation = 'relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.1))

model.add(Conv2D(64, 3, activation = 'relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(10, activation = 'softmax'))
model.summary()

NameError: name 'Sequential' is not defined

In [6]:
model.score(test_X.reshape(-1, 784), test_y)

0.9777

In [4]:
import pickle
with open('number.model', 'wb') as f:
    pickle.dump(model, f)

In [5]:
import numpy as np

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW) # Direct Show

if cap.isOpened():
    while True:
        ret, img = cap.read() # True, 이미지배열(넘파일 배열) 또는 False, None
        if not ret: break

        # 컬러 -> 회색조 -> 이진화(반전)
        g_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        _, bin_img = cv2.threshold(g_img, 110, 255, cv2.THRESH_BINARY_INV) # 2진화, 반전
        # 객체의 컨투어 찾기
        contours, h = cv2.findContours(bin_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        # 컨투어를 이용해서 이미지의 ROI 추출, 크기 변환
        for contour in contours:
            (x,y), radius = cv2.minEnclosingCircle(contour)
            if radius < 5: continue # 반지름이 5보다 작은 객체는 처리 안함
            xs, xe = int(x-radius), int(x+radius)
            ys, ye = int(y-radius), int(y+radius)
            cv2.rectangle(bin_img, (xs, ys), (xe, ye), (200, 0, 0), 2)

            roi = bin_img[ys:ye, xs:xe]
            if roi.shape[0]*roi.shape[1] == 0: continue
            roi = cv2.resize(roi, dsize=(24, 24))
            A = np.zeros((28, 28))
            A[2:-2, 2:-2] = roi
            A = A.reshape(-1, 784)
            num = model.predict(A)
            cv2.putText(bin_img, str(num), (xs, ys), cv2.FONT_HERSHEY_PLAIN, 2, (200,0,0))
        # 모델에 입력해서 예측
        # 예측한 숫자를 화면에 표시
        cv2.imshow("Image", bin_img)

        if cv2.waitKey(1) & 0xFF == 27:  # ESC
            break
else:
    print("camera not opened")

cap.release()
cv2.destroyAllWindows()